This is a supplementary Jupyter notebook for the paper "Shanks bias in function fields".

In [1]:
from ff import *

$L$-functions for $\mu$ and $\lambda$-bias.

In [2]:
def lfunc_mu(m, prec=None):
    """
    L-function for mu-bias, given by L_mu_m = 1 / L_chi_m.

    If precision is None, return as a rational function.
    If precision is given, return as a power series in u
    """
    L_chi_m = DirichletCharacterFFQuadratic(m).lfunc()
    if prec is None:
        K = L_chi_m.parent().fraction_field()
        L_mu = 1 / K(L_chi_m)
    else:
        R2.<u> = PowerSeriesRing(ZZ, default_prec=prec)
        L_mu = 1 / R2(L_chi_m)
    return L_mu


def lfunc_mu_cum(m, prec=None):
    """
    Cumulative L-function for mu, given by L_mu_cum_m = (L_mu - 1) / (1 - u)
    """
    L_mu = lfunc_mu(m, prec)
    return (L_mu - 1) / (1 - L_mu.parent().gen())


def lfunc_lambda(m, prec=None):
    """
    L-function for lambda-bias, given by
    L_lambda_m = \prod_{P | m} (1 - u^{2*deg(P)}) / (1 - q * u^2) / dirichlet_l_quad_char(m)

    If precision is None, return as a rational function.
    If precision is given, return as a power series in u
    """
    chi_m = DirichletCharacterFFQuadratic(m)
    L_chi_m = chi_m.lfunc()
    q = chi_m.q()
    u = L_chi_m.parent().gen()
    if prec is None:
        K = L_chi_m.parent().fraction_field()
        L_lambda = 1 / K(L_chi_m)
    else:
        R2.<u> = PowerSeriesRing(ZZ, default_prec=prec)
        L_lambda = 1 / R2(L_chi_m)
    for P, _ in m.factor():
        L_lambda *= 1 - u ^ (2 * P.degree())
    L_lambda /= (1 - q * u ^ 2)
    return L_lambda


def lfunc_lambda_cum(m, prec=None):
    """
    L-function for the cumulative lambda-bias, given by
    (L_lambda - 1) / (1 - u)
    """
    L_lambda = lfunc_lambda(m, prec)
    return (L_lambda - 1) / (1 - L_lambda.parent().gen())


Function to approximate density by counting up to certain degrees.

In [3]:
import polars as pl

def densities_table(m, M, n_max_list):
    L_mu = lfunc_mu(m, prec=M+1)
    L_mu_cum = lfunc_mu_cum(m, prec=M+1)
    L_lambda = lfunc_lambda(m, prec=M+1)
    L_lambda_cum = lfunc_lambda_cum(m, prec=M+1)
    rows = []
    for n_max in n_max_list:
        denom = float(n_max)
        delta_lambda      = sum(1 for n in range(1, n_max + 1) if L_lambda[n]      > 0) / denom
        delta_lambda_cum  = sum(1 for n in range(1, n_max + 1) if L_lambda_cum[n]  > 0) / denom
        delta_mu          = sum(1 for n in range(1, n_max + 1) if L_mu[n]          > 0) / denom
        delta_mu_cum      = sum(1 for n in range(1, n_max + 1) if L_mu_cum[n]      > 0) / denom
        rows.append({
            "n_max": n_max,
            "delta_lambda": delta_lambda,
            "delta_lambda_cum": delta_lambda_cum,
            "delta_mu": delta_mu,
            "delta_mu_cum": delta_mu_cum,
        })

    # Build a Polars DataFrame from a list of dicts
    df = pl.from_dicts(rows)
    return df

Approximate integral for density.

In [4]:
import numpy as np
from scipy.integrate import quad

def area(a, b, c, tol=1e-6):
    # Area of the set \{(x, y) \in [0, 2\pi]^2 | a sin(x) + b sin(y) > c}
    # Normalized by (2\pi)^2
    if b == 0:
        raise ValueError("b must be non-zero (swap roles of x and y otherwise).")
    # quick exclusions
    if c >= abs(a) + abs(b):
        return 0.0
    if c <= -abs(a) - abs(b):
        return 1.0
    
    def ell(d):
        if d >= 1.0:
            return 0.0
        if d <= -1.0:
            return 2*np.pi
        return 2*np.arccos(d)
    
    integrand = lambda x: ell((c - a*np.sin(x))/b)
    area_val, _ = quad(integrand, 0.0, 2*np.pi, epsabs=tol, epsrel=tol)
    return area_val / (2 * np.pi) ** 2

Degree 3 examples.

Example 1: $m_1(T) = T^3 + T + 4 \in \mathbb{F}_5[T]$ satisfies GSH.

In [5]:
R5.<t> = GF(5)['t']
m1 = t^3 + t + 4
chi_m1 = DirichletCharacterFFQuadratic(m1)
L_chi_m1 = chi_m1.lfunc()

print("Dirichlet L-function for chi_m1:")
print(L_chi_m1)

print("L-function for mobius bias:")
print(lfunc_mu(m1))

print("L-function for cumulative mobius bias:")
print(lfunc_mu_cum(m1))

print("L-function for lambda bias:")
print(lfunc_lambda(m1))

print("L-function for cumulative lambda bias:")
print(lfunc_lambda_cum(m1))

Dirichlet L-function for chi_m1:
5*u^2 + 3*u + 1
L-function for mobius bias:
1/5/(u^2 + 3/5*u + 1/5)
L-function for cumulative mobius bias:
(u^2 + 3/5*u)/(u^3 - 2/5*u^2 - 2/5*u - 1/5)
L-function for lambda bias:
(1/25*u^6 - 1/25)/(u^4 + 3/5*u^3 - 3/25*u - 1/25)
L-function for cumulative lambda bias:
(-1/25*u^6 + u^4 + 3/5*u^3 - 3/25*u)/(u^5 - 2/5*u^4 - 3/5*u^3 - 3/25*u^2 + 2/25*u + 1/25)


In [6]:
df = densities_table(m1, 10000, [10, 100, 1000, 10000])
print(df)

shape: (4, 5)
┌───────┬──────────────┬──────────────────┬──────────┬──────────────┐
│ n_max ┆ delta_lambda ┆ delta_lambda_cum ┆ delta_mu ┆ delta_mu_cum │
│ ---   ┆ ---          ┆ ---              ┆ ---      ┆ ---          │
│ i64   ┆ f64          ┆ f64              ┆ f64      ┆ f64          │
╞═══════╪══════════════╪══════════════════╪══════════╪══════════════╡
│ 10    ┆ 0.6          ┆ 0.6              ┆ 0.5      ┆ 0.4          │
│ 100   ┆ 0.62         ┆ 0.68             ┆ 0.49     ┆ 0.49         │
│ 1000  ┆ 0.616        ┆ 0.69             ┆ 0.499    ┆ 0.5          │
│ 10000 ┆ 0.6168       ┆ 0.6891           ┆ 0.4999   ┆ 0.4999       │
└───────┴──────────────┴──────────────────┴──────────┴──────────────┘


Explicit density using trigonometric functions:

In [7]:
def a(q, costheta):
    # a = (1 - q^(-3)) / sqrt(1 - 2 * q^(-3) * cos(6 * theta) + q^(-6))
    cos3theta = 4 * costheta**3 - 3 * costheta
    cos6theta = 2 * cos3theta**2 - 1
    return (1 - q**(-3)) / sqrt(1 - 2 * q**(-3) * cos6theta + q**(-6))

def a_even(q, costheta):
    # a_even = a * sqrt(1 - 2 * q^(-1/2) * cos(theta) + q^(-1)) * (q + q^(1/2) * cos(theta)) / (q - 1)
    return a(q, costheta) * sqrt(1 - 2 * q**(-1/2) * costheta + q**(-1)) * (q + q**(1/2) * costheta) / (q - 1)

def a_odd(q, costheta):
    # a_odd = a * sqrt(1 - 2 * q^(-1/2) * cos(theta) + q^(-1)) * (q * cos(theta) + q^(1/2)) / (q - 1)
    return a(q, costheta) * sqrt(1 - 2 * q**(-1/2) * costheta + q**(-1)) * (q * costheta + q**(1/2)) / (q - 1)

costheta = - 3 / sqrt(20)
q = 5
a_ = a(q, costheta)
a_even_ = a_even(q, costheta)
a_odd_ = a_odd(q, costheta)
print("a:", a_, float(a_))
print("a_even:", a_even_, float(a_even_))
print("a_odd:", a_odd_, float(a_odd_))

delta_lambda = 1 / 2 + (arcsin(a_) + arcsin(a_ * costheta)) / (2 * pi)
delta_lambda_cum = 1 / 2 + (pi / 2  + arcsin(a_odd_)) / (2 * pi)  # a_even_ > 1
print(f"delta_lambda: {float(delta_lambda):.6f}")
print(f"delta_lambda_cum: {float(delta_lambda_cum):.6f}")

a: 31/54*sqrt(3) 0.9943254636043554
a_even: 217/144*sqrt(3)*sqrt(1/5) 1.1672741473986241
a_odd: -31/144*sqrt(5)*sqrt(3)*sqrt(1/5) -0.3728720488516333
delta_lambda: 0.616823
delta_lambda_cum: 0.689187


Example 2: $m_2(T) = T^3 - T + 1 \in \mathbb{F}_3[T]$, where GSH does not hold.

In [8]:
R3.<t> = GF(3)['t']
m2 = t^3 - t + 1
chi_m2 = DirichletCharacterFFQuadratic(m2)
L_chi_m2 = chi_m2.lfunc()

print("Dirichlet L-function for chi_m2:")
print(L_chi_m2)

print("L-function for mobius bias:")
print(lfunc_mu(m2))

print("L-function for cumulative mobius bias:")
print(lfunc_mu_cum(m2))

print("L-function for lambda bias:")
print(lfunc_lambda(m2))

print("L-function for cumulative lambda bias:")
print(lfunc_lambda_cum(m2))

Dirichlet L-function for chi_m2:
3*u^2 - 3*u + 1
L-function for mobius bias:
1/3/(u^2 - u + 1/3)
L-function for cumulative mobius bias:
u/(u^2 - u + 1/3)
L-function for lambda bias:
(1/9*u^6 - 1/9)/(u^4 - u^3 + 1/3*u - 1/9)
L-function for cumulative lambda bias:
(-1/9*u^6 + u^4 - u^3 + 1/3*u)/(u^5 - 2*u^4 + u^3 + 1/3*u^2 - 4/9*u + 1/9)


In [9]:
df = densities_table(m2, 10000, [10, 100, 1000, 10000])
print(df)  # Densities are 3/4, 1, 5/12, 5/12

shape: (4, 5)
┌───────┬──────────────┬──────────────────┬──────────┬──────────────┐
│ n_max ┆ delta_lambda ┆ delta_lambda_cum ┆ delta_mu ┆ delta_mu_cum │
│ ---   ┆ ---          ┆ ---              ┆ ---      ┆ ---          │
│ i64   ┆ f64          ┆ f64              ┆ f64      ┆ f64          │
╞═══════╪══════════════╪══════════════════╪══════════╪══════════════╡
│ 10    ┆ 0.8          ┆ 1.0              ┆ 0.4      ┆ 0.5          │
│ 100   ┆ 0.76         ┆ 1.0              ┆ 0.44     ┆ 0.44         │
│ 1000  ┆ 0.751        ┆ 1.0              ┆ 0.419    ┆ 0.419        │
│ 10000 ┆ 0.7501       ┆ 1.0              ┆ 0.4169   ┆ 0.4169       │
└───────┴──────────────┴──────────────────┴──────────┴──────────────┘


Higher degree example: $m = (T^2 + T + 1) (T^3 + 2T + 1) \in \mathbb{F}_3[T]$.

In [10]:
R3.<t> = GF(3)['t']
m = (t^2 + 1) * (t^3 + 2 * t + 1)
chi_m = DirichletCharacterFFQuadratic(m)
L_chi_m = chi_m.lfunc()

print("Dirichlet L-function for chi_m:")
print(L_chi_m)

print("L-function for mobius bias:")
print(lfunc_mu(m))

print("L-function for cumulative mobius bias:")
print(lfunc_mu_cum(m))

print("L-function for lambda bias:")
print(lfunc_lambda(m))

print("L-function for cumulative lambda bias:")
print(lfunc_lambda_cum(m))

Dirichlet L-function for chi_m:
9*u^4 + 3*u^3 + 4*u^2 + u + 1
L-function for mobius bias:
1/9/(u^4 + 1/3*u^3 + 4/9*u^2 + 1/9*u + 1/9)
L-function for cumulative mobius bias:
(u^4 + 1/3*u^3 + 4/9*u^2 + 1/9*u)/(u^5 - 2/3*u^4 + 1/9*u^3 - 1/3*u^2 - 1/9)
L-function for lambda bias:
(-1/27*u^10 + 1/27*u^6 + 1/27*u^4 - 1/27)/(u^6 + 1/3*u^5 + 1/9*u^4 - 1/27*u^2 - 1/27*u - 1/27)
L-function for cumulative lambda bias:
(1/27*u^10 + 26/27*u^6 + 1/3*u^5 + 2/27*u^4 - 1/27*u^2 - 1/27*u)/(u^7 - 2/3*u^6 - 2/9*u^5 - 1/9*u^4 - 1/27*u^3 + 1/27)


Approximate density by counting up to certain degrees:

In [11]:
df = densities_table(m, 10000, [10, 100, 1000, 10000])
print(df)

shape: (4, 5)
┌───────┬──────────────┬──────────────────┬──────────┬──────────────┐
│ n_max ┆ delta_lambda ┆ delta_lambda_cum ┆ delta_mu ┆ delta_mu_cum │
│ ---   ┆ ---          ┆ ---              ┆ ---      ┆ ---          │
│ i64   ┆ f64          ┆ f64              ┆ f64      ┆ f64          │
╞═══════╪══════════════╪══════════════════╪══════════╪══════════════╡
│ 10    ┆ 0.7          ┆ 0.6              ┆ 0.5      ┆ 0.5          │
│ 100   ┆ 0.55         ┆ 0.71             ┆ 0.55     ┆ 0.48         │
│ 1000  ┆ 0.582        ┆ 0.736            ┆ 0.5      ┆ 0.501        │
│ 10000 ┆ 0.5849       ┆ 0.7382           ┆ 0.5005   ┆ 0.4991       │
└───────┴──────────────┴──────────────────┴──────────┴──────────────┘


Explicit $\lambda$-density:

In [12]:
cos1 = 1 / sqrt(12)
cos2 = -1 / sqrt(3)
cos21 = 2 * cos1^2 - 1
cos22 = 2 * cos2^2 - 1
cos31 = 4 * cos1^3 - 3 * cos1
cos32 = 4 * cos2^3 - 3 * cos2
cos41 = 2 * cos21^2 - 1
cos42 = 2 * cos22^2 - 1
cos61 = 2 * cos31^2 - 1
cos62 = 2 * cos32^2 - 1
sin1sq = 1 - cos1^2
sin2sq = 1 - cos2^2
Cm = (1 - 3 ** (-2)) * (1 - 3 ** (-3)) / (2 ** 2 * sin1sq * sin2sq)
e_even = 1 + cos1 * cos2
e_odd = cos1 + cos2

alpha_even = Cm * e_even
alpha_odd = Cm * e_odd
Alpha_even = Cm * ((3/2) * e_even + (sqrt(3) / 2) * e_odd)
Alpha_odd = Cm * ((3/2) * e_odd + (sqrt(3) / 2) * e_even)
print("Cm:", Cm, float(Cm))
print("alpha_even:", alpha_even, float(alpha_even))
print("alpha_odd:", alpha_odd, float(alpha_odd))
print("alpha_even':", Alpha_even, float(Alpha_even))
print("alpha_odd':", Alpha_odd, float(Alpha_odd))

beta1 = (2 * sqrt(3) / 11) * sqrt(1 - 2 * 3^(-2) * cos41 + 3^(-4)) * sqrt(1 - 2 * 3^(-3) * cos61 + 3^(-6))
beta2 = (sqrt(3) / 4) * sqrt(1 - 2 * 3^(-2) * cos42 + 3^(-4)) * sqrt(1 - 2 * 3^(-3) * cos62 + 3^(-6))
print("beta1:", beta1, float(beta1))
print("beta2:", beta2, float(beta2))

Beta1 = beta1 / sqrt(1 - 2 * 3^(-1/2) * cos1 + 3^(-1))
Beta2 = beta2 / sqrt(1 - 2 * 3^(-1/2) * cos2 + 3^(-1))
print("beta1':", Beta1, float(Beta1))
print("beta2':", Beta2, float(Beta2))


Cm: 104/297 0.3501683501683502
alpha_even: 260/891 0.29180695847362514
alpha_odd: -52/891*sqrt(3) -0.10108489561569205
alpha_even': 104/297 0.3501683501683502
alpha_odd': 52/891*sqrt(3) 0.10108489561569205
beta1: 40/297*sqrt(5)*sqrt(3)*sqrt(1/3) 0.30115393636360804
beta2: 2/27*sqrt(19)*sqrt(3)*sqrt(2/3) 0.4566232594791834
beta1': 40/297*sqrt(5)*sqrt(3)*sqrt(1/3) 0.30115393636360804
beta2': 1/27*sqrt(19)*sqrt(3)*sqrt(2)*sqrt(2/3) 0.32288140322523506


In [13]:
delta_lambda = 1 / 2 + 1 / 2 * (area(float(beta1), float(beta2), -float(alpha_odd)) - area(float(beta1), float(beta2), float(alpha_even)))
print(f"delta_lambda = {delta_lambda:.6f}")
delta_lambda_cum = 1 / 2 + 1 / 2 * (area(float(Beta1), float(Beta2), -float(Alpha_odd)) - area(float(Beta1), float(Beta2), float(Alpha_even)))
print(f"delta_lambda_cum = {delta_lambda_cum:.6f}")

delta_lambda = 0.584867
delta_lambda_cum = 0.739345
